In [9]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from typing import List, Tuple, Dict

dataset_path = "/home/kaba/development/lightfm_sandbox/ml-1m/"
raw_data_df = pd.read_csv(f"{dataset_path}ml100k.csv")

In [10]:
# 必要な７種の変数を用意する。
all_user_ids = sorted(raw_data_df['user_id'].unique())
all_item_ids = sorted(raw_data_df['item_id'].unique())

# ユーザー特徴量のカラム名
user_features_columns = [col for col in raw_data_df.columns if col.startswith('user_') and col != 'user_id']
unique_user_features_df = pd.unique(pd.concat([raw_data_df[col] for col in user_features_columns], ignore_index=True))
unique_user_features_list = unique_user_features_df.tolist()

# アイテム特徴量のカラム名
item_features_columns = [col for col in raw_data_df.columns if col.startswith('item_') and col != 'item_id']
unique_item_features_df = pd.unique(pd.concat([raw_data_df[col] for col in item_features_columns], ignore_index=True))
unique_item_features_list = unique_item_features_df.tolist()

data = list(zip(raw_data_df['user_id'], raw_data_df['item_id']))


In [16]:
print(all_user_ids)

[np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32), np.int64(33), np.int64(34), np.int64(35), np.int64(36), np.int64(37), np.int64(38), np.int64(39), np.int64(40), np.int64(41), np.int64(42), np.int64(43), np.int64(44), np.int64(45), np.int64(46), np.int64(47), np.int64(48), np.int64(49), np.int64(50), np.int64(51), np.int64(52), np.int64(53), np.int64(54), np.int64(55), np.int64(56), np.int64(57), np.int64(58), np.int64(59), np.int64(60), np.int64(61), np.int64(62), np.int64(63), np.int64(64), np.int64(65), np.int64(66), np.int64(67), np.int64(68), np.int64(69), np.int64(70), np.int64(71), np.int64(72), 

In [18]:
def build_feature_data(df: pd.DataFrame) -> List[Tuple[int, Dict]]:
    """カテゴリ変数は0.5、連続変数は0-1に正規化して特徴量辞書を生成"""
    result = []
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    continuous_cols = df.select_dtypes(include=['int', 'float']).columns.tolist()

    # user_id, item_idは正規化する必要がないため除外（あれば）
    for col_to_exclude in ['user_id', 'item_id']:
        if col_to_exclude in categorical_cols:
            categorical_cols.remove(col_to_exclude)

    # 連続値を0-1スケーリング
    if continuous_cols:
        scaler = MinMaxScaler()
        df[continuous_cols] = scaler.fit_transform(df[continuous_cols])

    for idx, row in df.iterrows():
        features = {}

        # カテゴリ変数
        for col in categorical_cols:
            val = row[col]
            if pd.notnull(val):
                features[f"{col}_{val}"] = 0.5

        # 連続値
        for col in continuous_cols:
            features[col] = row[col]

        result.append((idx, features))

    return result


In [19]:
# ユーザー特徴量抽出
user_df = raw_data_df[user_features_columns]
user_features_data = build_feature_data(user_df)

# アイテム特徴量抽出
item_df = raw_data_df[item_features_columns]
item_features_data = build_feature_data(item_df)

/tmp/ipykernel_64925/1373302462.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[continuous_cols] = scaler.fit_transform(df[continuous_cols])
/home/kaba/development/lightfm_sandbox/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:686: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/kaba/development/lightfm_sandbox/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:706: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/tmp/ipykernel_64925/1373302462.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

In [21]:
user_features_data

[(0,
  {'user_gender_M': 0.5,
   'user_occupation_writer': 0.5,
   'user_zip_code_55105': 0.5,
   'user_age': 0.6363636363636364}),
 (1,
  {'user_gender_F': 0.5,
   'user_occupation_executive': 0.5,
   'user_zip_code_00000': 0.5,
   'user_age': 0.48484848484848486}),
 (2,
  {'user_gender_M': 0.5,
   'user_occupation_writer': 0.5,
   'user_zip_code_40206': 0.5,
   'user_age': 0.2727272727272727}),
 (3,
  {'user_gender_M': 0.5,
   'user_occupation_technician': 0.5,
   'user_zip_code_80525': 0.5,
   'user_age': 0.3181818181818182}),
 (4,
  {'user_gender_M': 0.5,
   'user_occupation_educator': 0.5,
   'user_zip_code_55113': 0.5,
   'user_age': 0.6060606060606061}),
 (5,
  {'user_gender_M': 0.5,
   'user_occupation_executive': 0.5,
   'user_zip_code_01581': 0.5,
   'user_age': 0.5606060606060607}),
 (6,
  {'user_gender_M': 0.5,
   'user_occupation_engineer': 0.5,
   'user_zip_code_17110': 0.5,
   'user_age': 0.36363636363636365}),
 (7,
  {'user_gender_F': 0.5,
   'user_occupation_librarian'

In [14]:
from lightfm.data import Dataset

dataset = Dataset()

# 全てのユーザー、アイテムをリストアップ。
dataset.fit(users=all_user_ids, items=all_item_ids, user_features=unique_user_features_list, item_features=unique_item_features_list)
print(dataset)

# weightは不要のため＿として記述。
interactions, _ = dataset.build_interactions(data=data)
print('interactions: ', interactions)

user_features = dataset.build_user_features(user_features_data)
item_features = dataset.build_item_features(item_features_data)


interactions:  <COOrdinate sparse matrix of dtype 'int32'
	with 100000 stored elements and shape (943, 1682)>
  Coords	Values
  (195, 241)	1
  (185, 301)	1
  (21, 376)	1
  (243, 50)	1
  (165, 345)	1
  (297, 473)	1
  (114, 264)	1
  (252, 464)	1
  (304, 450)	1
  (5, 85)	1
  (61, 256)	1
  (285, 1013)	1
  (199, 221)	1
  (209, 39)	1
  (223, 28)	1
  (302, 784)	1
  (121, 386)	1
  (193, 273)	1
  (290, 1041)	1
  (233, 1183)	1
  (118, 391)	1
  (166, 485)	1
  (298, 143)	1
  (290, 117)	1
  (307, 0)	1
  :	:
  (536, 442)	1
  (617, 627)	1
  (486, 290)	1
  (112, 974)	1
  (942, 390)	1
  (863, 684)	1
  (749, 322)	1
  (278, 63)	1
  (645, 749)	1
  (653, 369)	1
  (616, 581)	1
  (912, 689)	1
  (659, 228)	1
  (420, 497)	1
  (494, 1090)	1
  (805, 420)	1
  (675, 537)	1
  (720, 261)	1
  (912, 208)	1
  (377, 77)	1
  (879, 475)	1
  (715, 203)	1
  (275, 1089)	1
  (12, 224)	1
  (11, 202)	1


ValueError: user id 0 not in user id mappings.

In [15]:
print(user_features_data)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
from lightfm import LightFM

# モデルの作成
model = LightFM(no_components=100, loss="bpr", random_state=123)

# 学習
res = model.fit(interactions=interactions, user_features=user_features, item_features=item_features)

res


In [ ]:
from lightfm.cross_validation import random_train_test_split

# interactionsを引数にとり、
train_interactions, test_interactions = random_train_test_split(interactions=interactions, test_percentage=0.2)

print(train_interactions, test_interactions)

<COOrdinate sparse matrix of dtype 'float32'
	with 1 stored elements and shape (3, 3)>
  Coords	Values
  (2, 2)	1.0 <COOrdinate sparse matrix of dtype 'float32'
	with 1 stored elements and shape (3, 3)>
  Coords	Values
  (0, 1)	1.0


In [ ]:
# modelのevaluation

from lightfm.evaluation import precision_at_k


precision_at_k = precision_at_k(model=model, test_interactions=test_interactions, train_interactions=train_interactions, k=10, user_features=all_user_features, item_features=all_item_features)

precision_at_k

AttributeError: 'list' object has no attribute 'tocsr'